# QuickStarter to the tournament

##Basic step and workflow:

0 - Using this notebook

1 - Download data

2 - Explore data

3 - Choose and train a model

4 - Scoring

5 - Make prediction

6 - Submit

---

# 0 - Using this notebook 

To execute the cell press `shift+enter`. 

Follow the steps and login with your Google account.

In [16]:
#%pip install xgboost

In [17]:
# Lib & Dependencies
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
import requests
from scipy import stats

print("Libraries import successful!")

Libraries import successful!


# 1 - Download data

We will provide you with two dataset

- Training_data will be use to train your model.
- Hackathon_data will be use to make your prediciton.


There is three target you need to provide prediction on: target_r, target_g, target_b.




In [18]:
# Data Download (may take a few minutes depending on your network)
train_datalink_X = 'https://tournament.crunchdao.com/data/X_train.csv'  
train_datalink_y = 'https://tournament.crunchdao.com/data/y_train.csv' 
hackathon_data_link = 'https://tournament.crunchdao.com/data/X_test.csv' 

# Data for training
train_data = pd.read_csv(train_datalink_X)
# Data for which you will submit your prediction
test_data = pd.read_csv(hackathon_data_link)
# Targets use for your supervised trainning
train_targets = pd.read_csv(train_datalink_y)

# If you don't want to work with time series
train_data = train_data.drop(columns=['id'])
test_data = test_data.drop(columns=['id'])

KeyboardInterrupt: 

In [ ]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [ ]:
train_data.isnull().sum().any() > 0

False

In [ ]:
train_targets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167289 entries, 0 to 1167288
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   target_r  1167289 non-null  float64
 1   target_g  1167289 non-null  float64
 2   target_b  1167289 non-null  float64
dtypes: float64(3)
memory usage: 26.7 MB


In [ ]:
test_data.isnull().sum().any() > 0

False

# 2 - Explore Data

Observe your data and prepare carefully what you will give to your model for training.

In [23]:
display(train_data)

,Moons,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13,Feature_14,Feature_15,Feature_16,Feature_17,Feature_18,Feature_19,Feature_20,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25,Feature_26,Feature_27,Feature_28,Feature_29,Feature_30,Feature_31,Feature_32,Feature_33,Feature_34,Feature_35,Feature_36,Feature_37,Feature_38,Feature_39,Feature_40,Feature_41,Feature_42,Feature_43,Feature_44,Feature_45,Feature_46,Feature_47,Feature_48,Feature_49
0,0,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.00,0.00,0.25,0.75,0.50,0.50,0.50,0.00,0.00,0.50,1.00,0.75,0.75,0.50,0.00,0.25,0.50,1.00,1.00,0.75,0.75,0.00,0.00,0.25,0.75,1.00,1.00,0.75,0.00,0.25,1.00,1.00,0.75,0.50,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75
1,0,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.50,0.50,0.75,0.75,0.50,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.25,0.25,0.75,0.75,0.75,0.50,0.50,0.50,0.50,1.00,0.75,0.75,0.75,0.50,0.50,0.25,0.50,0.75,1.00,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.25,0.25,0.25,0.25
2,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.50,0.75,1.00,0.00,0.00,0.25,0.00,0.50,0.50,0.50,0.00,0.00,0.25,0.50,0.50,0.50,0.25,0.00,0.00,0.00,0.00,0.25,0.50,0.50,0.25,0.00,0.00,0.75,0.75,0.25,0.50,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.25,0.25,0.25,0.00,0.00,0.00,0.25,0.25,0.25,0.25,0.25,0.00,0.00,0.25,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.50,0.25,0.25,0.25,0.00,0.25,0.50,0.25,0.50,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.75,0.50,1.00,1.00,1.00,1.00,1.00,1.00,0.75,1.00,1.00,1.00,1.00,1.00,0.75,0.75,0.75,1.00,1.00,1.00,1.00,1.00,0.75,1.00,1.00,1.00,1.00,0.75,0.25,0.50,1.00,1.00,1.00,1.00,1.00,1.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167284,493,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.50,0.25,0.75,0.50,0.75,1.00,0.50,1.00,1.00,1.00,0.00,1.00,1.00,0.75,0.50,0.50,0.75,0.75,0.75,0.75,0.75,0.50,0.50,0.25,0.75,0.50,0.75,0.75,0.25,0.25,1.00,0.25,0.50,0.50,0.75,1.00,1.00,1.00,1.00,1.00,1.00,1.00
1167285,493,0.50,0.50,0.50,0.50,0.50,0.50,0.50,1.00,1.00,0.75,0.50,0.50,0.75,0.50,0.75,0.75,0.75,0.25,0.25,0.75,0.50,1.00,1.00,1.00,0.75,0.50,0.50,0.50,1.00,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.75,0.25,0.25,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50
1167286,493,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.75,0.75,0.50,0.00,0.75,0.50,1.00,0.50,0.50,0.25,0.25,0.75,0.50,0.75,0.50,0.50,0.25,0.25,0.50,0.50,0.50,0.75,0.75,0.50,0.25,0.25,0.50,0.50,0.50,0.75,0.00,0.25,0.75,0.75,0.75,0.50,0.50,0.50,0.50,0.50,0.50,0.50
1167287,493,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.75,0.75,0.50,0.75,1.00,0.75,0.75,0.75,0.75,0.75,0.75,1.00,0.50,0.75,1.00,1.00,1.00,1.00,1.00,1.00,0.75,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.75,0.75,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50


# 3 - Choose models

In [ ]:
# model trial
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import VotingRegressor

In [20]:
SEED = 41

# Random Forest Regressor Initialization
rf_reg = RandomForestRegressor(n_estimators=1000, max_depth=5, min_samples_split=100, min_samples_leaf=50, min_weight_fraction_leaf=0.4, ccp_alpha=0.5, max_samples=0.8,   
                                n_jobs=-1, random_state=SEED)


In [21]:
# Extra Tress Regressor Initialization
et_reg = ExtraTreesRegressor(n_estimators=1000, max_depth=5, min_samples_split=100, min_samples_leaf=50, min_weight_fraction_leaf=0.4, ccp_alpha=0.5, max_samples=0.8,
                            bootstrap=True, n_jobs=-1, random_state=SEED)


In [25]:
# Xtra Gradient Boosting Regressor Initialization
xgb_reg = xgb.XGBRegressor(max_depth=10, min_child_weight=4, subsample=0.5, learning_rate=0.2, n_estimators=1000, colsample_bytree=0.5)


# 4 - Scoring: Spearman's rank correlation of your predictions vs the targets

In [ ]:
def scorer(y_test, y_pred):
    score = (stats.spearmanr(y_test, y_pred)*100)[0]
    print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))

In [26]:
# Voting Regressor Initialization
models = [('rf', rf_reg), ('et', et_reg), ('xgb', xgb_reg)]
vote_reg = VotingRegressor(estimators=models, verbose=True)

def best_reg(data, target):
    X, y = data, target

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=SEED)    

    vote_reg.fit(X_train, y_train)

    preds = vote_reg.predict(X_test)

    scorer(y_test, preds)

    return vote_reg

In [27]:
# Score Model

model_target_r = best_reg(train_data, train_targets.target_r)
model_target_g = best_reg(train_data, train_targets.target_g)
model_target_b = best_reg(train_data, train_targets.target_b)

[Voting] ....................... (1 of 3) Processing rf, total= 4.9min
[Voting] ....................... (2 of 3) Processing et, total= 3.0min
[Voting] ...................... (3 of 3) Processing xgb, total=76.5min
Score as calculated for the leader board (っಠ‿ಠ)っ 0.2002038241238327


KeyboardInterrupt: 

# 5 - Make prediction on the 3 targets

When you feel like your model is accurate enough it's time to predict the target and submit your results.

Repeat the operation on the three targets, concatenate the answers and submit.

**WARNING** 1/ Keep the raw order identical.

**WARNING** 2/ Be sure that your columns are named target_r, target-g and
 target_b.

**WARNING** 3/ Your prediction need to be between 0 and 1.

**WARNING** 4/ Don't submit constant values.

In [ ]:
prediction = pd.DataFrame()
prediction['target_r'] = model_target_r.predict(test_data)
prediction['target_g'] = model_target_g.predict(test_data)
prediction['target_b'] = model_target_b.predict(test_data)

In [ ]:
prediction

,target_r,target_g,target_b
0,0.461819,0.461819,0.461819
1,0.504563,0.504563,0.504563
2,0.554732,0.554732,0.554732
3,0.513043,0.513043,0.513043
4,0.542707,0.542707,0.542707
...,...,...,...
29093,0.439170,0.439170,0.439170
29094,0.451500,0.451500,0.451500
29095,0.470727,0.470727,0.470727
29096,0.553274,0.553274,0.553274


# 6 - Submit predictions

Past <u>your</u> API key here. You received it by email upon registration and can retrieve it in your dashboard.


In [ ]:
API_KEY = "API_KEY" # <- HERE

r = requests.post("https://tournament.crunchdao.com/api/v2/submissions",
    files = {
        "file": ("x", prediction.to_csv().encode('ascii'))
    },
    data = {
        "apiKey": API_KEY
    },
)

if r.status_code == 200:
    print("Submission submitted.")
elif r.status_code == 423:
    print("ERR: Submissions are close")
    print("You can only submit during rounds eg: Friday 7pm GMT+1 to Sunday midnight GMT+1.")
    print("Or the server is currently crunching the submitted files, please wait some time before retrying.")
elif r.status_code == 422:
    print("ERR: API Key is missing or empty")
    print("Did you forget to fill the API_KEY variable?")
elif r.status_code == 400:
    print("ERR: The file must not be empty")
    print("You have send a empty file.")
elif r.status_code == 401:
    print("ERR: Your email hasn't been verified")
    print("Please verify your email or contact a cruncher.")
elif r.status_code == 409:
    print("ERR: Duplicate submission")
    print("Your work has already been submitted with the same exact results, if you think that this a false positive, contact a cruncher.")
    print("MD5 collision probability: 1/2^128 (source: https://stackoverflow.com/a/288519/7292958)")
elif r.status_code == 429:
    print("ERR: Too many submissions")
else:
    print("ERR: Server returned: " + str(r.status_code))
    print("Ouch! It seems that we were not expecting this kind of result from the server, if the probleme persist, contact a cruncher.")

Submission submitted.


_______
_______
# Submission #2

## Updated following
    * Additional XGBoostRegressor Parameters (tree_method='gpu_hist')
    * Need to Figure out -- LEARNING TASK PARAMETERS
    * Increase test_size to 0.2


In [45]:
# Xtra Gradient Boosting Regressor Initialization
from locale import normalize


xgb_reg = xgb.XGBRegressor(max_depth=10, min_child_weight=4, subsample=0.5, learning_rate=0.2, n_estimators=2000, colsample_bytree=0.5, colsample_bylevel=0.5, colsample_bynode=0.5,
                            tree_method="gpu_hist", num_parallel_tree=5, 
                            objective='reg:squaredlogerror', eval_metric='rmsle')
# sample_type='weighted', normalize_type='forest', feature_selector='thrifty', top_k=25

In [46]:
# Voting Regressor Initialization
models = [('rf', rf_reg), ('et', et_reg), ('xgb', xgb_reg)]
vote_reg = VotingRegressor(estimators=models, verbose=True)

def best_reg(data, target):
    X, y = data, target

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=SEED)    

    vote_reg.fit(X_train, y_train)

    preds = vote_reg.predict(X_test)

    scorer(y_test, preds)

    return vote_reg

In [47]:
# Score Model

model_target_r = best_reg(train_data, train_targets.target_r)
model_target_g = best_reg(train_data, train_targets.target_g)
model_target_b = best_reg(train_data, train_targets.target_b)

[Voting] ....................... (1 of 3) Processing rf, total= 4.8min
[Voting] ....................... (2 of 3) Processing et, total= 3.0min
[Voting] ...................... (3 of 3) Processing xgb, total=26.3min
Score as calculated for the leader board (っಠ‿ಠ)っ 0.22007197709279727
[Voting] ....................... (1 of 3) Processing rf, total= 4.7min
[Voting] ....................... (2 of 3) Processing et, total= 2.8min
[Voting] ...................... (3 of 3) Processing xgb, total=25.5min
Score as calculated for the leader board (っಠ‿ಠ)っ 0.2666509274119054
[Voting] ....................... (1 of 3) Processing rf, total= 4.7min
[Voting] ....................... (2 of 3) Processing et, total= 2.8min
[Voting] ...................... (3 of 3) Processing xgb, total=25.7min
Score as calculated for the leader board (っಠ‿ಠ)っ 0.29175514746915276


In [48]:
prediction = pd.DataFrame()
prediction['target_r'] = model_target_r.predict(test_data)
prediction['target_g'] = model_target_g.predict(test_data)
prediction['target_b'] = model_target_b.predict(test_data)

In [49]:
prediction

,target_r,target_g,target_b
0,0.506232,0.506232,0.506232
1,0.406685,0.406685,0.406685
2,0.461826,0.461826,0.461826
3,0.475063,0.475063,0.475063
4,0.492084,0.492084,0.492084
...,...,...,...
29093,0.442926,0.442926,0.442926
29094,0.472523,0.472523,0.472523
29095,0.413318,0.413318,0.413318
29096,0.489415,0.489415,0.489415


In [50]:
API_KEY = "API_KEY"

r = requests.post("https://tournament.crunchdao.com/api/v2/submissions",
    files = {
        "file": ("x", prediction.to_csv().encode('ascii'))
    },
    data = {
        "apiKey": API_KEY
    },
)

if r.status_code == 200:
    print("Submission submitted.")
elif r.status_code == 423:
    print("ERR: Submissions are close")
    print("You can only submit during rounds eg: Friday 7pm GMT+1 to Sunday midnight GMT+1.")
    print("Or the server is currently crunching the submitted files, please wait some time before retrying.")
elif r.status_code == 422:
    print("ERR: API Key is missing or empty")
    print("Did you forget to fill the API_KEY variable?")
elif r.status_code == 400:
    print("ERR: The file must not be empty")
    print("You have send a empty file.")
elif r.status_code == 401:
    print("ERR: Your email hasn't been verified")
    print("Please verify your email or contact a cruncher.")
elif r.status_code == 409:
    print("ERR: Duplicate submission")
    print("Your work has already been submitted with the same exact results, if you think that this a false positive, contact a cruncher.")
    print("MD5 collision probability: 1/2^128 (source: https://stackoverflow.com/a/288519/7292958)")
elif r.status_code == 429:
    print("ERR: Too many submissions")
else:
    print("ERR: Server returned: " + str(r.status_code))
    print("Ouch! It seems that we were not expecting this kind of result from the server, if the probleme persist, contact a cruncher.")

Submission submitted.
